In [ ]:
import json, re,  os, time, random, time
from bs4 import BeautifulSoup   
import pandas as pd 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC  
from config import *
from datetime import datetime, date, timedelta
import pymongo
import logging 
from ast import literal_eval
config=''
with open(r'C:\Users\timko\source\repos\PythonBoiler\CodeGenerators\config.json') as f:
    config = json.loads(f.read())  

<a id="id"></a>

In [ ]:
#with open('config.log','w') as f:
#    pass

In [ ]:
logging.basicConfig(filename='config.log', level=logging.INFO, format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p') 
#logging.getLogger().setLevel(logging.INFO)
logger=logging.getLogger('config')
logger.debug(f'DEBUG')
logger.info(f'INFO')

In [ ]:
# db.test.ensureIndex({name: 1, nodes: 1}, {unique: true, dropDups: true}) 

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn) 
db = client.indeed 
db.posts.insert_many( [{'a':'1', 'b':'2'},{'a':'1', 'b':'3'},{'a':'2', 'b':'3'}] ) 
 
df = pd.DataFrame(list(db.posts.find())) 
df.drop_duplicates(subset='a', inplace=True) 
db.posts.delete_many({})
db.posts.insert_many( df.to_dict('records')  )
 

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn) 
db = client.dcum
collection = db.posts 

In [ ]:
options = Options()
options.add_argument("--start-minimized")  
options.add_argument("--window-size=1020,980")  
# driver = webdriver.Chrome(executable_path=ChromeDriverManager(log_level=0, version='114.0.5735.90').install() ,options=options)   
driver = webdriver.Chrome(executable_path=r"C:\bom\drivers\117\chromedriver.exe" ,options=options)     

In [ ]:

def get_id(s):
    return re.search('list/(\d+)\.page',s).groups(1)[0]
def get_elements(url):
    lod=[]
    driver.get(url)  
    logger.info(f'[page      ] {url}') 
    topics = driver.find_elements_by_xpath("//*[contains(@class, 'topic-list')]//*[@class='subject-line']//a")
    urls = [t.get_attribute("href") for t in topics] 
    logger.info(f'[topics     ] {len(urls)}')
    for turl in urls:  
        driver.get(turl) 
        #time.sleep(.15)
        logger.info(f'[processing] {turl}')
        try:
            subject = WebDriverWait(driver, timeout=2).until(lambda driver: driver.find_elements_by_xpath("//*[contains(@class, 'category-heading-topic')]"))  
            logger.info(f'[subject   ] {subject[0].text}')
            # subject = driver.find_elements_by_xpath("//*[contains(@class, 'category-heading-topic')]")
            postbody = driver.find_elements_by_xpath("//*[contains(@class, 'postbody')]")
            postinfo = driver.find_elements_by_xpath("//*[contains(@class, 'postinfo')]")[0].text
            d={  
                'topic_url':url,
                'subject':subject[0].text,
                'href':turl,
                'date': re.search('(\d{1,2}/\d{1,2}/\d{4})',postinfo).groups(1)[0],
                'postbody':[re.sub('\t','', p.get_attribute('innerHTML'))  for p in postbody]
            } 
            lod.append(d) 
        except BaseException as exception:
            logging.exception(f"Exception: {type(exception).__name__} {exception}") 
    return lod

#get_elements('https://www.dcurbanmom.com/jforum/forums/show/50.page') 

In [ ]:
# collection.delete_many({})

In [ ]:
start = 25
pages = 10
for i in range(2450, 2500, 25): 
    url = f'https://www.dcurbanmom.com/jforum/forums/show/{i}/50.page' 
    print(url) 
    lod = get_elements(url)  
    # df = pd.DataFrame(lod)   df.to_dict('records') 
    collection.insert_many( lod )
 


array([False, False, False, ..., False, False, False])

In [84]:
df = pd.DataFrame(list(collection.find()))
df['date'] = df['date'].apply(lambda s: datetime.strptime(s, '%m/%d/%Y')) 
df['href'] = df['href'].astype(str) 
df['body'] = df['postbody'].apply(lambda s: re.sub('\n',' ',BeautifulSoup(s[0], "lxml").text))
df['subody']=df['subject']+ ' ' +df['body']
df = df.astype(str).drop_duplicates(subset='href', keep="first") 
df.sort_values(by='date', ascending=False, inplace=True) 
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2393 entries, 2064 to 1253
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   _id        2393 non-null   object
 1   topic_url  2393 non-null   object
 2   subject    2393 non-null   object
 3   href       2393 non-null   object
 4   date       2393 non-null   object
 5   postbody   2393 non-null   object
 6   body       2393 non-null   object
 7   subody     2393 non-null   object
dtypes: object(8)
memory usage: 168.3+ KB


In [76]:

dff = df.loc[df['body'].str.contains(r'(http)', case=False)]
# dff = dff.loc[~df['postbody'].str.contains(r'(#########)', case=False)]
dff.sort_values(by='date', ascending=False, inplace=True) 
print( len(dff) )
for i,r in dff.iterrows():
    print(r['subject'] + ' ( '+ str(r['date']).split(' ')[0] + ' )\n' + r['href'].split(';')[0] )

624
Thoughts on this House in CCDC? ( 2023-10-20 )
https://www.dcurbanmom.com/jforum/posts/list/1162971.page
Two Old Town Alexandria houses worth coveting ( 2023-10-18 )
https://www.dcurbanmom.com/jforum/posts/list/1162613.page
Real Estate commissions could be on chopping block - lawsuit ( 2023-10-18 )
https://www.dcurbanmom.com/jforum/posts/list/1162573.page
Fox News’s Bret Baier hopes to sell for a record price ( 2023-10-18 )
https://www.dcurbanmom.com/jforum/posts/list/1162681.page
Real Estate commissions could be on chopping block - lawsuit ( 2023-10-18 )
https://www.dcurbanmom.com/jforum/posts/list/1162573.page
really beautiful MCM house in Baltimore with 2 pools (indoor and outdoor) ( 2023-10-17 )
https://www.dcurbanmom.com/jforum/posts/list/1162458.page
really beautiful MCM house in Baltimore with 2 pools (indoor and outdoor) ( 2023-10-17 )
https://www.dcurbanmom.com/jforum/posts/list/1162458.page
Annie Leibovitz's apartment on Central Park West is for sale ( 2023-10-17 )
https:

c:\Users\timko\anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
<ipython-input-76-63c550b54acc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff.sort_values(by='date', ascending=False, inplace=True)


In [ ]:
dff = df.loc[df['subject'].str.contains(r'(rates|insan|prices|market|predict|when will|how high|economy|interest)', case=False)]
dff = dff.loc[~df['subject'].str.contains(r'(features|closing|structure|home|house|days|negotiable|condo|HELOC|Cap Hill Project)', case=False)]
dff.sort_values(by='date', ascending=False, inplace=True) 
print( len(dff) )
for i,r in dff.iterrows():
    print(r['subject'] + ' ( '+ str(r['date']).split(' ')[0] + ' )\n' + r['href'].split(';')[0] )
 

In [ ]:
df = pd.DataFrame(lod)   
collection.delete_many({})
collection.insert_many( df.to_dict('records') )

In [ ]:
df = pd.DataFrame(lod)   
df.to_csv(rf'c:\temp\dcumre-{pages}.csv', index=False)

In [ ]:

df=pd.read_csv(r'c:\temp\dcumre.csv',converters={"postbody": literal_eval})
df['id'] = df['href'].apply(get_id) 
postbody=df.iloc[3]['postbody'] 
postbody


In [ ]:
options = Options()
options.add_argument("--start-minimized")  
options.add_argument("--window-size=1220,980")  
# driver = webdriver.Chrome(executable_path=ChromeDriverManager(log_level=0, version='114.0.5735.90').install() ,options=options)   
driver = webdriver.Chrome(executable_path=r"C:\bom\drivers\117\chromedriver.exe" ,options=options)   

driver.get(f"https://dayman.cyber-balance.com/jira/login.jsp")  
driver.find_element(By.XPATH, '//input[contains(@id, "user")]').send_keys(config['user'])
driver.find_element(By.XPATH, '//input[contains(@id, "pass")]').send_keys(config['pass']) 
driver.find_element(By.XPATH, '//input[contains(@id, "submit")]').click() 

In [ ]:
driver.get(f'https://dayman.cyber-balance.com/jira/secure/Dashboard.jspa')
eles = driver.find_elements_by_xpath("//td[@class='fixVersions']//a[contains(text(),'7.5')]/../..//a[@class='issue-link']")
all_issues=[e.get_attribute("href") for e in eles]
all_issues=list(set(all_issues))
eles = driver.find_elements_by_xpath("//a[contains(text(),'BOD')]")
issues=[e.get_attribute("href") for e in eles]
issues=list(set(issues))

In [ ]:

issues

In [ ]:
log()

In [ ]:
for issue in all_issues[:]: 
    driver.get(issue)  

In [ ]:
def log():
    for issue in issues[:]: 
        print(issue) 
        driver.get(issue)  
        try:
            ele = WebDriverWait(driver, timeout=2).until(lambda d: d.find_elements_by_xpath("//span[contains(text(),'Start Progress')]")) 
            if len(ele) > 0:
                ele[0].click() 
                driver.get(issue) 
                time.sleep(1)
        except Exception as e:
            print(e) 
        ele = WebDriverWait(driver, timeout=2).until(lambda d: d.find_element_by_xpath("//a[@id='opsbar-operations_more']"))
        ele.click()
        ele = WebDriverWait(driver, timeout=2).until(lambda d: d.find_element_by_xpath("//span[contains(text(),'Log work')]"))
        ele.click()
        ele = WebDriverWait(driver, timeout=2).until(lambda d: d.find_element_by_xpath("//*[@id='log-work-time-logged']"))
        ele.send_keys('20m') 
        ele = driver.find_elements_by_xpath("//*[@id='log-work-submit']")
        ele[0].click()
        time.sleep(5)




In [ ]:
def subo():
    for issue in issues[:]:  
        driver.get(issue)   
        time.sleep(5)  
        print(issue)   
        try:
            ele = WebDriverWait(driver, timeout=5).until(lambda d: d.find_elements_by_xpath("//span[contains(text(),'Resolve Issue')]")) 
            if len(ele) > 0:
                ele[0].click()   
                ele = WebDriverWait(driver, timeout=2).until(lambda d: d.find_element_by_xpath("//input[@id='issue-workflow-transition-submit']")) 
                ele.click()   
        except Exception as e:
            print(e) 

        try:
            ele = WebDriverWait(driver, timeout=5).until(lambda d: d.find_elements_by_xpath("//span[contains(text(),'Ready To Test')]")) 
            if len(ele) > 0:
                ele[0].click()    
                ele = WebDriverWait(driver, timeout=2).until(lambda d: d.find_element_by_xpath("//input[@id='issue-workflow-transition-submit']")) 
                ele.click()   
        except Exception as e:
            print(e) 